In [1]:
import os
import torch
import modules.strings as strings
from pprint import pprint
import modules.augmenting as augmenting

# torch.set_printoptions(threshold=160000)
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
# problema librerie che usano OpenMP, due versioni di OpenMP sono caricate nello stesso processo, ad esempio da PyTorch e da NumPy

#### Load data, do data augmentation

In [2]:
root_path = strings.find(os.getcwd(), "data_images")
print(root_path)

/home/user/DAML-project/data_images


In [3]:
for name in ["train", "valid", "test"]:
    datasets = augmenting.augment_dataset(os.path.join(root_path, name), times=2)
    pprint(name, strings.classes_nums(datasets[0]))

    results, targets = augmenting.map_datasets(datasets)
    print(results.shape)
    print(targets.shape)
    torch.save({"Features": results, "Targets": targets}, os.path.join(root_path, f"{name}.pt"))

{'Adenocarcinoma': 195,
 'LargeCellCarcinoma': 115,
 'Noncancer': 148,
 'SquamousCellCarcinoma': 155}
torch.Size([1839, 512])
torch.Size([1839])
{'Adenocarcinoma': 23,
 'LargeCellCarcinoma': 21,
 'Noncancer': 13,
 'SquamousCellCarcinoma': 15}
torch.Size([216, 512])
torch.Size([216])
{'Adenocarcinoma': 120,
 'LargeCellCarcinoma': 51,
 'Noncancer': 54,
 'SquamousCellCarcinoma': 90}
torch.Size([945, 512])
torch.Size([945])
